# Exemple PT1 Profes, orientatiu.

## Pregunta 1. Importació del dataset.

### 1.1 On obtenir el dataset. 
Dataset:
https://analisi.transparenciacatalunya.cat/Salut/COVID-19-Seq-enciaci-de-variants/k4w3-3i4f

Instrucció prèvia, per reduir tamany CSV, només volem els del 2022.
```bash
cat seqvariants.csv | tail -n+2 | grep -E ',2022,' seqvariants.csv > seqvariants2022.csv
```

Alternativa, grep per comarques.

### 1.2 Descripció
Les dades publicades en aquest conjunt provenen del nou Sistema de Vigilància d’Infeccions de Catalunya (SIVIC), del Departament de Salut, que integra les funcions de vigilància epidemiològica de la grip, de la COVID-19 i d’altres virus respiratoris, amb l'objectiu de monitorar aquests virus, recollir els canvis de tendència i millorar la caracterització microbiològica dels virus circulants.

### 1.3 Nom de les columnes, descripció i tipus.
setmana_epidemiologica	Text Pla        Observacions
any		                Nombre          
data_inici		        Data & Temps    Podem usar-lo a la consulta 7.
data_final		        Data & Temps
codi_regio		        Text Pla        Id Regio (comarca)
nom_regio		        Text Pla        Nom Regio (comarca)
codi_ambit		        Text Pla
nom_ambit		        Text Pla
variant		            Text Pla
sexe		            Text Pla        Gènere. Pot ser categòric.
grup_edat		        Text Pla        Pot ser categòric.
index_socioeconomic		Text Pla        Pot ser categòric.
recompte		        Nombre


In [9]:
# Imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import copy

# Q1 - Llegim el CSV.
csv_file_path = 'seqvariants2022.csv'
df_seqvariants_covid: pd.DataFrame = pd.read_csv(csv_file_path, sep=",")
df_seqvariants_covid.head(20)

,setmana_epidemiologica,any,data_inici,data_final,codi_regio,nom_regio,codi_ambit,nom_ambit,variant,sexe,grup_edat,index_socioeconomic,recompte
0,28,2022,11/07/2022,17/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.4),Dona,15 a 44,4,1
1,28,2022,11/07/2022,17/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.4),Home,80 o més,2,1
2,28,2022,11/07/2022,17/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.5),Dona,70 a 79,4,1
3,28,2022,11/07/2022,17/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.5),Dona,80 o més,4,2
4,28,2022,11/07/2022,17/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.5),Home,70 a 79,4,1
5,27,2022,04/07/2022,10/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.2),Dona,15 a 44,1,1
6,27,2022,04/07/2022,10/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.2),Dona,15 a 44,3,1
7,27,2022,04/07/2022,10/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.2),Dona,45 a 59,2,1
8,27,2022,04/07/2022,10/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.2),Dona,60 a 69,4,1
9,27,2022,04/07/2022,10/07/2022,78,BARCELONA,7803,BARCELONA CIUTAT,Omicron (BA.2),Dona,80 o més,2,3


### Verifiquem els dtypes, que els hagi agafat correctament.

#### Tots els ha agafat bé menys els data_inici i data_fi.
He d'investigar com convertir les dates de format llatí a format ISO aaaa-mm-dd

In [10]:
df_seqvariants_covid.dtypes

setmana_epidemiologica     int64
any                        int64
data_inici                object
data_final                object
codi_regio                 int64
nom_regio                 object
codi_ambit                 int64
nom_ambit                 object
variant                   object
sexe                      object
grup_edat                 object
index_socioeconomic        int64
recompte                   int64
dtype: object

## Pregunta 2 - Arreglant el dataset.

1. Cada fila és una observació. 
   1. Sí, es una observació de les dades d'una setmana en una regió, de cada variant del Covid19 (moltes són omicron) 
2. Cada columna és una variable.
   1. En gairebé tots els casos sí, excepte **codi_regio = nom_regio**, **codi_ambit = nom_ambit**
   2. Segurament podriem eliminar els codis.
3. Cada cel·la conté només una dada.
   1. Sí. Només hem de veure com fer per agrupar algunes dades.

In [12]:
# Fem còpia de seguretat.
df_seqvariants_covid_copy = copy.deepcopy(df_seqvariants_covid)

df_seqvariants_covid = df_seqvariants_covid.drop(columns=['codi_regio','codi_ambit'])
df_seqvariants_covid

KeyError: "['codi_regio', 'codi_ambit'] not found in axis"